In [77]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [78]:
# Data directories
train_dir = 'dataset/seg_train/seg_train'
val_dir = 'dataset/seg_val/seg_val'
test_dir = 'dataset/seg_pred/seg_pred'

In [79]:
# Image data generators with data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)
val_test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

val_generator = val_test_datagen.flow_from_directory(
    val_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

test_generator = val_test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

Found 14034 images belonging to 6 classes.
Found 3000 images belonging to 6 classes.
Found 0 images belonging to 0 classes.


In [80]:
# Model architecture
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(6, activation='softmax')  # 6 categories
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


c:\Users\vignesh\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [81]:
# Callbacks for early stopping and model checkpointing
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ModelCheckpoint('best_model.keras', save_best_only=True)
]


In [82]:
# Model training
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=val_generator,
    validation_steps=val_generator.samples // val_generator.batch_size,
    epochs=20,
    callbacks=callbacks
)


Epoch 1/20


c:\Users\vignesh\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


438/438 ━━━━━━━━━━━━━━━━━━━━ 401s 904ms/step - accuracy: 0.5253 - loss: 1.2322 - val_accuracy: 0.7221 - val_loss: 0.7328
Epoch 2/20
  1/438 ━━━━━━━━━━━━━━━━━━━━ 3:59 547ms/step - accuracy: 0.6250 - loss: 0.8547

c:\Users\vignesh\AppData\Local\Programs\Python\Python311\Lib\contextlib.py:155: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


438/438 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.6250 - loss: 0.8547 - val_accuracy: 0.7500 - val_loss: 0.4870
Epoch 3/20
438/438 ━━━━━━━━━━━━━━━━━━━━ 331s 754ms/step - accuracy: 0.7169 - loss: 0.7536 - val_accuracy: 0.7665 - val_loss: 0.6581
Epoch 4/20
438/438 ━━━━━━━━━━━━━━━━━━━━ 1s 286us/step - accuracy: 0.7500 - loss: 0.6558 - val_accuracy: 0.8333 - val_loss: 0.5645
Epoch 5/20
438/438 ━━━━━━━━━━━━━━━━━━━━ 309s 702ms/step - accuracy: 0.7617 - loss: 0.6486 - val_accuracy: 0.8202 - val_loss: 0.5273
Epoch 6/20
438/438 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.6875 - loss: 0.8887 - val_accuracy: 0.9583 - val_loss: 0.2442
Epoch 7/20
438/438 ━━━━━━━━━━━━━━━━━━━━ 294s 667ms/step - accuracy: 0.8001 - loss: 0.5487 - val_accuracy: 0.7789 - val_loss: 0.5960
Epoch 8/20
438/438 ━━━━━━━━━━━━━━━━━━━━ 1s 337us/step - accuracy: 0.7500 - loss: 0.6767 - val_accuracy: 0.7500 - val_loss: 0.5580
Epoch 9/20
438/438 ━━━━━━━━━━━━━━━━━━━━ 275s 626ms/step - accuracy: 0.8256 - loss: 0.4951 - val_acc

In [83]:
# Model evaluation
try:
    test_loss, test_acc = model.evaluate(test_generator, steps=test_generator.samples // test_generator.batch_size)
    print(f'Test Accuracy: {test_acc:.4f}')
except Exception as e:
    print("Error during model evaluation:", e)

print("Number of samples in test generator:", test_generator.samples)



Error during model evaluation: Must provide at least one structure
Number of samples in test generator: 0


In [84]:
# Save the model
model.save('final_model.keras')

In [85]:
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model

# Load the trained model
model = load_model('final_model.keras')

# Define the path to your input image
input_image_path = 'dataset\\seg_pred\\seg_pred\\24326.jpg'  # Update with the path to your input image

# Load and preprocess the input image
img = image.load_img(input_image_path, target_size=(150, 150))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.  # Normalize the image

# Predict the class probabilities
predictions = model.predict(img_array)

# Get the predicted class label
predicted_class_index = np.argmax(predictions)
class_labels = ['buildings', 'forest', 'glacier', 'mountain', 'sea', 'street']  # Define your class labels
predicted_class_label = class_labels[predicted_class_index]

print("Predicted class:", predicted_class_label)
print("Predicted probabilities:", predictions)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 283ms/step
Predicted class: forest
Predicted probabilities: [[3.7310593e-04 9.9954790e-01 4.5007804e-05 9.6049553e-06 6.6517907e-07
  2.3572586e-05]]
